In [11]:
#from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.backend import set_session, get_session
 
from keras import backend as K
from tqdm import tqdm
import pickle
import cv2
import glob
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from keras.models import model_from_json, Sequential, Model, load_model

from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Lambda, ELU, Cropping2D, Dropout, BatchNormalization
from keras.layers.convolutional import Convolution2D, Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam
import matplotlib.pyplot as plt

from random import randint

In [12]:
ResNet50_model_for_dog_breed = ResNet50(weights='imagenet')

In [13]:
Res_model_for_adjusting_shape = ResNet50(weights='imagenet', include_top=False)

In [14]:
bottleneck_features = np.load('bottleneck_features/DogResnet50Data.npz')
train_Resnet = bottleneck_features['train']
valid_Resnet = bottleneck_features['valid']
test_Resnet = bottleneck_features['test']

In [15]:

Resnet_Model = Sequential()
Resnet_Model.add(GlobalAveragePooling2D(input_shape=train_Resnet.shape[1:]))
Resnet_Model.add(Dense(133,activation='softmax'))
Resnet_Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
Resnet_Model.load_weights('saved_models/weights.best.Resnet.hdf5')

In [16]:
def extract_Resnet50(tensor):
    return ResNet50(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

In [21]:

#define generic function for pre-processing images into 4d tensor as input for CNN
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)


#predicts the dog breed based on the pretrained ResNet50 models with weights from imagenet
def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model_for_dog_breed.predict(img))

### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    print(prediction)
    return ((prediction <= 268) & (prediction >= 151))

# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

def Resnet_predict_breed(img_path):
    # extract bottleneck features
    y = path_to_tensor(img_path)
    y = preprocess_input(y)
    print(y.shape)
    x = Res_model_for_adjusting_shape.predict(y)
    #bottleneck_feature = extract_Resnet50(path_to_tensor(img_path))
    print(x.shape)
    # obtain predicted vector
    predicted_vector = Resnet_Model.predict(x)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]


def get_correct_prenom(word, vowels):
    if word[0].lower() in vowels:
            return "an"
    else:
        return "a"


def predict_image(img_path):
    vowels=["a","e","i","o","u"]
    #if a dog is detected in the image, return the predicted breed.
    if dog_detector(img_path)==True:
        predicted_breed=Resnet_predict_breed(img_path).rsplit('.',1)[1].replace("_", " ")
        prenom=get_correct_prenom(predicted_breed,vowels)
        return "The predicted dog breed is " + prenom + " "+ str(predicted_breed) + "."
    #if a human is detected in the image, return the resembling dog breed.
    if face_detector(img_path)==True:
        predicted_breed=Resnet_predict_breed(img_path).rsplit('.',1)[1].replace("_", " ")
        prenom=get_correct_prenom(predicted_breed,vowels)
        return "This photo looks like " + prenom + " "+ str(predicted_breed) + "."
    #if neither is detected in the image, provide output that indicates an error.
    else:
        return "No human or dog could be detected, please provide another picture."


In [22]:
import json
dog_names=[]
with open('data/dog_names.json') as json_file:
    dog_names = json.load(json_file)

In [23]:
img_path='static/Brittany_02625.jpg'

In [24]:

predict_image('static/Brittany_02625.jpg')

215
(1, 224, 224, 3)
(1, 1, 1, 2048)


'The predicted dog breed is a Brittany.'

In [25]:
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__
list(imports())

['builtins',
 'builtins',
 'keras.utils.np_utils',
 'numpy',
 'glob',
 'keras.preprocessing.image',
 'keras.backend',
 'pickle',
 'cv2',
 'tensorflow',
 'pandas',
 'os',
 'matplotlib.pyplot',
 'json',
 'types']

In [26]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to had
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

In [27]:
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

tqdm==4.11.2
tensorflow==1.3.0
pandas==0.23.3
numpy==1.18.4
matplotlib==2.1.0


In [28]:
import platform
print(platform.python_version())

3.6.3
